In [1]:
import spacy
import csv
import pandas as pd
import glob

from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

from numpy import nan

In [2]:
nlp = spacy.load("en_core_web_sm")

# modify tokenizer infix patterns for avoiding hyphenated words being split (several terms are hyphenated)

infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [3]:
def read_tsv(link):
    with open(link, "r", encoding="utf-8") as tsvfile:
        reader = csv.reader(tsvfile, delimiter="\t")
        return {row[0] for row in reader}

terms_lexicon = read_tsv("../extraction/swimming-terms_FINAL.tsv") # set of extracted terms

In [5]:
len(terms_lexicon)

2167

In [6]:
# open corpus, spacy it

corpus = []

for txt in glob.iglob("../corpus_txt/*.txt"):
    with open(txt, "r", encoding="utf-8") as f:
        doc = f.readlines()[0]
        corpus.append(doc)

In [7]:
nlp.max_length = 1500000
nlp.add_pipe(nlp.create_pipe('sentencizer'))
doc = nlp(" ".join(corpus), disable=['ner', 'parser'])

In [8]:
df = pd.DataFrame([item for sub in [[tok.text for tok in s]+['<eos>'] for s in doc.sents] for item in sub], columns=["token"]) # create df with a token in each cell, sentences separated by <eos>

df["lemma"] = [item for sub in [[token.lemma_.lower() if token.lemma_ != '-PRON-' else token.lower_ for token in s]+['<eos>'] for s in doc.sents] for item in sub] # create a column with lemmas

In [9]:
df # with lowercase and lemmitized tokens to allow 1-to-1 matching with terms

,token,lemma
0,J.,j.
1,Swimming,swimming
2,Research,research
3,",",","
4,Vol,vol
...,...,...
291036,http://www.wasa.asn.au/articles/article.asp?Gr...,http://www.wasa.asn.au/articles/article.asp?gr...
291037,",",","
291038,1999,1999
291039,.,.


In [10]:
# initially populate 3rd column with "O" tags, which will be changed later
# replace <eos> with nan values for handy exporting later

df["tag"] = df["token"].apply(lambda x: nan if x == "<eos>" else "O")
df["token"] = df["token"].apply(lambda x: nan if x == "<eos>" else x)

In [11]:
df

,token,lemma,tag
0,J.,j.,O
1,Swimming,swimming,O
2,Research,research,O
3,",",",",O
4,Vol,vol,O
...,...,...,...
291036,http://www.wasa.asn.au/articles/article.asp?Gr...,http://www.wasa.asn.au/articles/article.asp?gr...,O
291037,",",",",O
291038,1999,1999,O
291039,.,.,O


In [12]:
max_term_len = max([len(x.split(" ")) for x in terms_lexicon])
print(max_term_len)

6


In [13]:
# Creating a new df which contains the terms ngrams (n from 1 to 6)
# And the index of the first word of this ngram in the original df

# around 120 secs to run

ngram_df_entries = []

for i in range(1,max_term_len+1):
    for j in range(len(df)):
        if j+i > len(df):
            pass
        n_gram = ' '.join(df.iloc[j: j+i]["lemma"])
        if n_gram in terms_lexicon:
            ngram_df_entries.append([j, n_gram, i])

In [14]:
ngram_df = pd.DataFrame(columns=["starting_index", "ngram", "ngram_len"], data=ngram_df_entries)

In [15]:
ngram_df

,starting_index,ngram,ngram_len
0,263,stress-recovery,1
1,328,responsiveness,1
2,337,endurance,1
3,436,endurance,1
4,475,in-water,1
...,...,...,...
18472,71393,computation of lift and drag force,6
18473,75707,computation of lift and drag force,6
18474,135182,frequency of training and training session,6
18475,151181,computation of lift and drag force,6


In [16]:
# Filtering duplicates (keeping the biggest ngrams)

ngram_df["is_duplicate"] = ngram_df.duplicated(subset="starting_index", keep='last')

In [17]:
ngram_df = ngram_df.query('is_duplicate == False')

In [18]:
ngram_df

,starting_index,ngram,ngram_len,is_duplicate
0,263,stress-recovery,1,False
1,328,responsiveness,1,False
5,1068,hyperventilation,1,False
6,2186,responsiveness,1,False
7,2353,responsiveness,1,False
...,...,...,...,...
18472,71393,computation of lift and drag force,6,False
18473,75707,computation of lift and drag force,6,False
18474,135182,frequency of training and training session,6,False
18475,151181,computation of lift and drag force,6,False


In [19]:
for index, row in ngram_df.iterrows():
    for i in range(row.ngram_len):
        if i==0:
            tag = "B"
        else:
            tag = "I"
        df.at[row.starting_index+i,"tag"] = tag

## Exploration, saving, splitting dataset

In [20]:
df.tag.value_counts()

O    247732
I     15601
B     14105
Name: tag, dtype: int64

In [21]:
df.query('tag=="B"')

,token,lemma,tag
68,physiological,physiological,B
79,elite,elite,B
90,elite,elite,B
122,bronchial,bronchial,B
150,high,high,B
...,...,...,...
290765,training,training,B
290844,influence,influence,B
290989,Hand,hand,B
290990,force,force,B


In [22]:
# train, dev, test BIO dataset creation for sequence term tagger

train_len = int(len(df)/100*80)
dev_len = train_len+int(len(df)/100*10)

df.iloc[:train_len].to_csv("../corpus_bio/train.txt", sep=" ", header=False, index=False, columns=["token", "tag"])
df.iloc[train_len:dev_len].to_csv("../corpus_bio/dev.txt", sep=" ", header=False, index=False, columns=["token", "tag"])
df.iloc[dev_len:].to_csv("../corpus_bio/test.txt", sep=" ", header=False, index=False, columns=["token", "tag"])